# Deep Autoencoders

In this notebook, we will explore autoencoders to perform the following three tasks

- Data compression
- Data generation
- Data interpolation

We will be using [MNIST digit dataset](https://en.wikipedia.org/wiki/MNIST_database), which is easily available through [`torchvision.datasets`](https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.MNIST). 

### Setup

For this tutorial, we will need the following python packages 

1. numpy
2. matplotlib
3. torch
4. torchvision

Please follow the instructions [here](https://pytorch.org/) to install the last two libraries. 

In [ ]:
# basic imports
import numpy as np
import matplotlib.pyplot as plt
import math
import pathlib

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from matplotlib.lines import Line2D

# fix seed for reproducibility 
rng = np.random.RandomState(1)
torch.manual_seed(rng.randint(np.iinfo(int).max))

# it is a good practice to define `device` globally
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", device)
else:
    device = torch.device("cpu")
    print("No GPU -> using CPU:", device)

# to store images
img_dir = pathlib.path("./images")
if not img_dir.exists():
    img_dir.mkdir()

### Data

PyTorch has simple-to-use functions that downloads and loads the datasets. We will use these functions to streamline our deep learning pipeline.

Checkout other image datasets at [torch.datasets](https://pytorch.org/vision/stable/datasets.html).

In [ ]:
# download data in data folder. It will create this folder if it doesn't exist
torchvision.datasets.MNIST(root="./data/", download=True)

### Explore Data

We will be carrying out simple investigations as we did in the practical of denoising autoencoders. 
Specifically, we are interested in finding out the following :

How does the data look like?

- What is the `type` of data?
- What does each element of data represent?
- What are the constituent parts of each element?
- How is the image represented?
- What do we use to plot an image?
- How do we use the image in our model?

In [ ]:
# load data 
data = torchvision.datasets.MNIST(root="./data/", train=True) # only load training data

In [ ]:
# print("Type of data is the class\n", type(data))
# print("\nEach element of the data is\n", type(data[0]))
# print("\nA single element is\n", data[0],"\n\nfirst element is the image and the second element is the category")

### Image data

We know that the image data is present in `PIL.Image.Image` format. We need to convert it to array of pixel values to operate on it. We will also be displaying these images. `matplotlib.pyplot` has a simple function `plt.imshow()` to display matrix as an image.

In [ ]:
# img = transforms.ToTensor()(data[10][0])
# print(f"Raw image to tensor shape: {img.shape}")

# # To plot this, we need a 2D array, so we use squeeze_(0) to remove first dimension
# img = img.squeeze_(0)
# print(f"2D image tensor shape: {img.shape}")

# # how to display the image 
# plt.imshow(img)

In [ ]:
# visualize
n_samples = len(data)
n_vis = 20

nrows = 4
ncols = math.ceil(n_vis/nrows)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12,6), dpi=100)

idxs = np.random.randint(low=0, high=n_samples, size=n_vis)

for i, idx in enumerate(idxs):
    ax = axs[i%nrows, i//nrows]
    img = transforms.ToTensor()(data[idx][0]).squeeze_(0)
    ax.imshow(img)
    ax.axis('off')

_ = fig.suptitle(f"Original MNIST dataset")


In [ ]:
input_shape = img.shape

## Data Compression: Simple setup & Vanilla Deep Autoencoder

We are interested in finding a compact representation for digits in our dataset. 
We will choose our latent dimension to be a 2D plane so that it's easy to visualize. 
However, depending on the usecase (e.g, classification, semi-supervised classification, etc.), one might choose a higher dimensional latent space.

In this section, will be setting up our code to test it on various autoencoders discussed in the lessons. 
We will be using a vailla Autoencoder to build our necessary functions, and then we will reuse these functions with different autoencoders.

To do this, we will build an autoencoder that comprises of an encoder and a decoder. 

In the following block of code, we will be implementing a simple **deep autoencoder**.
We will be using linear layers to process our images, although you are free to chose the architecture you like.
It consists of two blocks -
- Encoder
- Decoder

We will be following the same pattern to implement other autoencoders in the rest of the tutorial.

In [ ]:
class DeepAutoencoder(nn.Module):
    def __init__(self, input_shape, latent_dim):
        super(DeepAutoencoder, self).__init__()
        
        self.input_shape = input_shape
        self.latent_dim = latent_dim
        self.in_dim = input_shape[0] * input_shape[1]
        
        # encoder 
        self.encoder = nn.Sequential(
            ## YOUR DESIGN OF ARCHITECTURE
        )
        
        # decoder 
        self.decoder = nn.Sequential(
            ## YOUR DESIGN OF ARCHITECTURE
            ## MAKE SURE LAST LAYER's OUTPUT IS IN THE CORRECT RANGE
        )
        
    
    def encode(self, x):
        # ANY PREPROCESSING ON x
        return self.encoder(x)
    
    def decode(self, z):
        x_hat = self.decoder(z)
        # ANY POST PROCESSING ON x_hat
        return x_hat

    def forward(self, x):
        z = self.encode(x)
        x_hat = self.decode(z)
        return z, x_hat

We need to write a function to fetch the memory statistics of our model.
While building different autoencoder models, it will be important to ensure they have all similar number of learnable parameters. This will ensure fair comparison of their performance.

In [ ]:
def mem_size(model):
    """
    Get model size in GB (as str: "N GB")
    """
    mem_params = sum(
        [param.nelement() * param.element_size() for param in model.parameters()]
    )
    mem_bufs = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])
    mem = mem_params + mem_bufs
    return f"{mem / 1e9:.4f} GB"

def num_params(model):
    """
    Print number of parameters in model's named children
    and total
    """
    s = "Number of parameters:\n"
    n_params = 0
    for name, child in model.named_children():
        n = sum(p.numel() for p in child.parameters())
        s += f"  • {name:<15}: {n}\n"
        n_params += n
    s += f"{'total':<19}: {n_params}"

    return s

def pp_model_summary(model):
    print(num_params(model))
    print(f"{'Total memory':<18} : {mem_size(model)}")

For fair comparison of different models, we need to make sure that they have the same parameters. The above two functions will come in handy to do that!

In [ ]:
# CHECK HOW MANY PARAMETERS DOES THE MODEL HAVE
# model_ae = DeepAutoencoder(input_shape, 2)
# pp_model_summary(model_ae)

### Dataset loading

PyTorch uses `torch.utils.data.Dataset` class to load data in parallel on multiple CPUs. It enables faster loading of the batch of data. It then uses `torch.utils.data.DataLoader` to combine these loaded data points together into a batch. These batches are then used as an input to the models.

For this reason, we need to customize `__len__` and `__getitem__` functions in the class. This will help us do any necessary preprocessing on the images before using them as an input to the model.

In [ ]:
# create a dataset 
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data 
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img = transforms.ToTensor()(self.data[index][0]).squeeze_(0)
        label = self.data[index][1] # to be used for visualization
        return img, label

### Loss 

We will create a function that will take following inputs:

1. An autoencoder instance
2. `torch.utils.data.DataLoader` instance 
3. loss function to compute loss based on the inputs 
4. optimizer (optional)

This function returns the mean loss on this data. We will be redesigning parts of this model further. 

In [ ]:
def process(model, data, loss_fn, optimizer=None):
    """
    Trains the `model` on `data`
    
    Args:
        model (nn.Module): model to process the data
        data (torchvision.datasets): MNIST dataset
        loss_fn (fn): how you want to compute the loss
        optimizer (torch.optim): Optimizer of choice 
    
    Returns:
        (float): mean loss 
    """
    
    n_samples = 0
    running_loss = 0
    for batch_img, _ in data:
        # transfer to GPU if avaiable
        x = batch_img.to(device)

        n_samples += x.shape[0]
        
        # compute loss 
        loss = loss_fn(model, x)
        
        # backward pass 
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item()
        
    return running_loss / n_samples
        

### Train

Finally, we will be running the training for `n_epochs` number of epochs. 
Each epoch consists of doing a backward pass on the subset of training data and evaluating the model on the validation data. 

**Note**: If one is concerned with good representation of the training data only (i.e., generalization outside training data is not of concern), we need not split our dataset into train and validation. 
However, in this tutorial, we will use the validation dataset to pick a model with the best generalization error. 

We will create a function `train` to conveniently call it with various models and loss functions that we can experiment with. 

In [ ]:
def train(model, loss_fn, epochs=30, append_title=""):
    """
    Trains the `model` using `process`
    
    Args:
        model (nn.Module): model to process the data
        data (torchvision.datasets): MNIST dataset
        optimizer (torch.optim): Optimizer of choice 
        process (fn): function defining how each epoch is handled
    
    Returns:
        best_model (nn.Module): returns the model with the best training loss 
        loss_plot (matplotlib.subplots): a plot for how training loss varied over the epochs
    """
    # create a model directory to store the best model
    model_dir = pathlib.Path("./models").resolve()
    if not model_dir.exists():
        model_dir.mkdir()

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

    n_epochs = epochs
    epoch_size=200
    batch_size=128

    best_train_loss = np.inf
    train_losses = []
    for epoch in range(n_epochs):
        print(f"@ epoch {epoch}", end="")

        # training loss
        idxs = rng.choice(len(data), epoch_size * batch_size, replace=True)
        train_data = torch.utils.data.DataLoader(Dataset([data[idx] for idx in idxs]), batch_size=batch_size, num_workers=4)
        train_loss = process(model, train_data, loss_fn, optimizer)
        
        # save
        if train_loss < best_train_loss:
            best_train_loss = train_loss 
            torch.save(model.state_dict(), model_dir / f"best_{model.__class__.__name__}_L_{model.latent_dim}{append_title}.ckpt")
    
        # log
        train_losses.append(train_loss)
        print(f"\ttrain_loss: {train_loss: .7f}")

    # load the best model
    model = model.__class__(model.input_shape, model.latent_dim)
    model.load_state_dict(torch.load(model_dir / f"best_{model.__class__.__name__}_L_{model.latent_dim}{append_title}.ckpt"))
    model = model.to(device)

    return model, train_losses


In [ ]:
# a function to create a string out of model; to be used as a title for visualizing
def get_model_title(model):
    """returns the string title for model"""
    return f"{model.__class__.__name__} (latent_dim={model.latent_dim})"


def visualize_losses(train_losses, title=""):
    """
    Visualize the loss trajectory 
    
    Args:
        train_losses (list): elements are float
        title (str): title to appear at the top of the figure
    """
    # plot losses
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(10,5), dpi=100)

    axs.plot(train_losses, color="#BDD9BF", marker="x", label="Train loss")
    axs.set_xlabel("Epochs", fontsize=20)
    axs.legend(prop={"size":15})

    # tick size
    for tick in axs.xaxis.get_major_ticks():
        tick.label.set_fontsize(15)

    for tick in axs.yaxis.get_major_ticks():
        tick.label.set_fontsize(15)
        
    fig.suptitle(title)

### Visualize Reconstructed images

Here we will write a function that can help us visualize the quality of reconstruction 


In [ ]:
def vis_reconstructed_images(model, append_title=""):
    """
    Randomly samples images and compare them with reconstructed imates usiing `model`
    
    Args:
        model (nn.Module): model to use to reconstruct the images
    """
    model = model.cpu()
    n_samples = len(data)
    n_vis = 20

    nrows = 4
    ncols = math.ceil(n_vis/nrows)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12,8), dpi=100)

    idxs = np.random.randint(low=0, high=n_samples, size=n_vis)

    for i, idx in enumerate(idxs):
        ax = axs[i%nrows, i//nrows]

        x = transforms.ToTensor()(data[idx][0]).squeeze_(0)

        with torch.no_grad():
            z, x_hat = model(x)
            x_hat = x_hat.squeeze_(0)

        filler =  np.ones((img.shape[0], 10))
        concat_img = np.concatenate((x, filler, x_hat), axis=1)

        ax.imshow(concat_img)
        ax.axis('off')

    title = f"{get_model_title(model)}{append_title}"
    _ = fig.suptitle(f"Original MNIST & recostructed images by {title} ")
    
    fig.savefig(f"images/recons_imgs_{title}.png")


### Visualize embeddings

Here we will write a function that can help us visualize 2D embeddings of points

In [ ]:
def get_latent_representations(model, n=1000, seed=1):
    """
    Randomly samples data from x_train, and returns their encoded representation using `model`
    
    Args:
        model (nn.Module): model to call `encode` on
        n (int): number of samples to use 
    
    Returns:
        points (list): encoded representation of random samples 
        labels (list): corresponding labels
    """
    model = model.cpu()
    n_samples = len(data)
     
    np.random.seed(seed)
    idxs = np.random.randint(low=0, high=n_samples, size=n)

    # encode
    points = []
    labels = []
    for idx in idxs:
        
        img, y = data[idx]
        img =  transforms.ToTensor()(img).squeeze_(0)
        
        with torch.no_grad():
            z = model.encode(img).squeeze_(0).numpy().tolist()

        points.append(z)
        labels.append(y)
    
    return points, labels


# use t-SNE for projecting to 2d: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html
def project_points_on_2D(points):
    """
    Returns the 2D projection of `points`
    
    Args:
        poitns (list): some high dimensional points 
    
    Returns:
        (list): 2D representation of the points 
    """
    if len(points[0]) <=2 :
        return list(points)

    from sklearn.manifold import TSNE  
    points = TSNE(n_components=2).fit_transform(np.array(points))
    return points.tolist()
    

def vis2D(model, n=6000, seed=1, append_title=""):
    """
    visualizes 2D representation of the points encoded using `model`.

    Args:
        model (nn.Module): model to call `encode` on
        n (int): number of samples to use     
    """
    points, labels = get_latent_representations(model, n, seed)
    
    if len(points[0]) > 2:
        points = project_points_on_2D(points)
    
    # visualize
    COLORS = ['#208ea3', '#62bb35', '#8d9f9b', '#e8384f', '#4178bc', '#fd817d', '#fdae33', '#a4c61a', '#37a862', '#7471f6', "#ea4e9d"]
    colormap = {i:c for i,c in enumerate(COLORS)}
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,8), dpi=100)

    x, y = zip(*points)
    colors = [colormap[l] for l in labels]
    ax.scatter(x, y, s=20, label=labels, c=colors)
    
    # legend
    legend = []
    for i, c in colormap.items():
        legend.append(Line2D([0, 1], [0, 0], color=c, label=f"{i}", linewidth=5))
    
    ax.legend(handles=legend, ncol=1, loc="upper right", fontsize=15, fancybox=True)
                          
     # title
    title = f"{get_model_title(model)}{append_title}"
    _ = fig.suptitle(f"Latent representation of digits by {title} ")
    fig.savefig(f"images/vis2d_{title}.png")
    

### Full setup

Finally, we only need to use the above defined functions in the following way to check the performance of these models.

In [ ]:
# our final setup (vanilla AE)
def reconstruction_loss(model, x):
    z, x_hat = model(x)
    return ## YOUR LOSS FUNCTION FOR RECONSTRUCTION GOES HERE

In [ ]:
model_deepae, losses = train(DeepAutoencoder(input_shape, 2), reconstruction_loss, epochs=30)

In [ ]:
# visualize
vis_reconstructed_images(model_deepae)
vis2D(model_deepae)

**Questions**
- What do we observe about the spatial arrangement of clusters?
- Are they will separated?
- What will it mean to pick a random point in the 2D space and use it as an input to the decoder?

### Data Generation: Variational Autoencoder (VAE)

In [ ]:
class VariationalAutoencoder(DeepAutoencoder):
    
    def __init__(self, input_shape, latent_dim):
        super().__init__(input_shape, latent_dim)
        
        # we will overwrite the encoder 
        self.encoder = nn.Sequential(
            ## YOUR ENCODER ARCHITECTURE 
            ## TRY TO KEEP IT SIMILAR TO DEEPAUTOENCODER
        )
        
        self.mu_head = nn.Sequential(
             ## YOUT ARCHITECTURE TO COMPUTE MU
        )
        
        self.log_var_head = nn.Sequential(
             ## YOUT ARCHITECTURE TO COMPUTE log_var
        )
        
    def encode_dist(self, x):
        ## YOUR CODE 
        return mu, log_var
    
    def sample(self, mu, log_var):
        # we sample from a normal distribution and use mu and log_var for appropriate transformation
        epsilon = torch.randn_like(mu)
        return mu + torch.exp(0.5*log_var) * epsilon
    
    def encode(self, x):
        return self.sample(*self.encode_dist(x))
        

In [ ]:
model = VariationalAutoencoder(input_shape, 2)

In [ ]:
pp_model_summary(model)

Note that our model has the same number of parameters as `model_deepae`

In [ ]:
def vae_loss(model, x, beta=1):
    """
    Computes vae loss
    """
    mu, log_var = model.encode_dist(x)
    x_hat = model.decode(model.sample(mu, log_var))
    
    ## CHECK THE SLIDES TO COMPUTE KL DIVERGENCE BETWEEN standard Normal and Normal with mu and var
    variational_loss = ## YOUR CODE GOES HERE
    
    return nn.BCELoss(reduction="sum")(x_hat, x) + beta * variational_loss

In [ ]:
model_vae = VariationalAutoencoder(input_shape, 2)


# some sources suggest warming up without KL before introducing KL term ... 
# print("Warming up without KL:\n")
# vae_loss_fn = lambda model, x: vae_loss(model, x, 0)
# model_vae, losses = train(model_vae, vae_loss_fn, epochs=10)

print("\ntraining with KL:\n")
beta = 1
vae_loss_fn = lambda model, x: vae_loss(model, x, beta)
model_vae, losses = train(model_vae, vae_loss_fn, epochs=30)

In [ ]:
append_title=f"kl_{beta:.2f}"
vis_reconstructed_images(model_vae, append_title=append_title)
vis2D(model_vae, append_title=append_title)

We will generate random points in the latent space to see how the decoder can be used to generate data points
we will use both of the above models to generate data. 

In [ ]:
def generate_data(model, append_title="", n_gen=20):
    n_gen = n_gen
    z = torch.randn((n_gen, model.latent_dim))
    
    with torch.no_grad():
        x_hat = model.decode(z)

    n_rows = 4
    n_cols = math.ceil(n_gen/n_rows)
    fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(12,8), dpi=100)
    for i in range(n_gen):
        ax = axs[i%n_rows, i//n_rows]
        ax.imshow(x_hat[i, :])
        ax.axis('off')
    
    title = f"{get_model_title(model)}{append_title}"
    _ = fig.suptitle(f"Decoded representations of random points in the latent space {title}")
    fig.savefig(f"images/rand_data_{title}.png")
    

In [ ]:
generate_data(model_vae, append_title=f"vae_{beta:0.2f}")
generate_data(model_deepae)

**Questions**
- Which model seems to be good at generating better data?


### Data Interpolation using VAE

Finally, we want to see how the interpolation results look like for different models. 
For this we will write a function to be called on different models.

In [ ]:
# pick some pairs of datapoints from the dataset
n_interpolate = 5
idxs = rng.randint(len(data), size=(n_interpolate, 2))
dataset = Dataset(data)
in1 = torch.cat([dataset[idx][0].unsqueeze_(0) for idx in idxs[:, 0]], dim=0)
in2 = torch.cat([dataset[idx][0].unsqueeze_(0) for idx in idxs[:, 1]])

In [ ]:
def interpolate(in1, in2, model, append_title):
    """
    Interpolates latent representations on `in1` and `in2`
    
    """
    alphas = np.arange(0, 1.01, 0.2)
    x_hats = []
    with torch.no_grad():
        z1 = model.encode(in1)
        z2 = model.encode(in2)
        for i, alpha in enumerate(alphas):
            z = (1- alpha) * z1 + alpha * z2
            x_hat = model.decode(z)
            x_hats.append(x_hat)
    
    # visualize
    fig, axs = plt.subplots(nrows=in1.shape[0], ncols=len(alphas) + 2, figsize=(12, 8), dpi=100)
    
    # plot originals 
    for j in range(in1.shape[0]):
        axs[j, 0].imshow(in1[j, :])
        axs[j, -1].imshow(in2[j, :])
        
        axs[j, 0].axis('off')
        axs[j, -1].axis('off')
    
    # plot decoded images
    for j in range(in1.shape[0]):
        for i, _ in enumerate(alphas):
            axs[j, i+1].imshow(x_hats[i][j, :])
            axs[j, i+1].axis('off')
    
    ## 
    title = f"{get_model_title(model)}{append_title}"
    _ = fig.suptitle(f"Data interpolation for {title}")
    fig.savefig(f"images/interpolated_{title}.png")
    

In [ ]:
interpolate(in1, in2, model_vae, append_title=f"vae_{beta:0.2f}")
interpolate(in1, in2, model_deepae)

**Questions**
- Can we say anything about the quality of data interpoalation for the two models?